In [ ]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from gtda.graphs import KNeighborsGraph
import itertools
from utils import GraphRNN_dataset, GraphRNN_DataSampler
import random

from importlib import reload
import matplotlib.pyplot as plt

from preprocessor import Preprocessor
from preprocessor import draw_network
from preprocessor import get_adj_from_plot

from preprocessor_final_data import Preprocessor
from preprocessor_final_data import draw_network
from preprocessor_final_data import get_adj_from_plot

import preprocessor_final_data 
import os
from torch_geometric.nn import GCNConv

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from Model3 import GraphConvolutionalNetwork

In [ ]:
print("Gathering data...")
flow_dataset = "../data/daily_county2county_2019_01_01.csv"
epi_dataset = "../data_epi/epidemiology.csv"
epi_dates = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
             "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
             "2020-06-21", "2020-06-22", "2020-06-23", "2020-06-24",
                "2020-06-25", "2020-06-26", "2020-06-27", "2020-06-28",
                "2020-06-29", "2020-06-30", "2020-07-01", "2020-07-02",
                "2020-07-03", "2020-07-04", "2020-07-05", "2020-07-06",
                "2020-07-07", "2020-07-08", "2020-07-09", "2020-07-10",
                "2020-07-11", "2020-07-12", "2020-07-13", "2020-07-14",
                "2020-07-15", "2020-07-16", "2020-07-17", "2020-07-18",
                "2020-07-19", "2020-07-20", "2020-07-21", "2020-07-22",
                "2020-07-23", "2020-07-24", "2020-07-25", "2020-07-26",
                "2020-07-27", "2020-07-28", "2020-07-29", "2020-07-30"
             ]

epi_dates_medium = [
    "2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
    "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
    "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
    "2020-06-21", "2020-06-22", "2020-06-23", "2020-06-24",
    "2020-06-25", "2020-06-26", "2020-06-27", "2020-06-28",
    "2020-06-29", "2020-06-30", "2020-07-01", "2020-07-02",
    "2020-07-03", "2020-07-04", "2020-07-05", "2020-07-06",
    "2020-07-07", "2020-07-08", "2020-07-09", "2020-07-10",
    "2020-07-11", "2020-07-12", "2020-07-13", "2020-07-14",
    "2020-07-15", "2020-07-16", "2020-07-17", "2020-07-18",
    "2020-07-19", "2020-07-20", "2020-07-21", "2020-07-22",
    "2020-07-23", "2020-07-24", "2020-07-25", "2020-07-26",
    "2020-07-27", "2020-07-28", "2020-07-29", "2020-07-30",
    "2020-07-31", "2020-08-01", "2020-08-02", "2020-08-03",
    "2020-08-04", "2020-08-05", "2020-08-06", "2020-08-07",
    "2020-08-08", "2020-08-09", "2020-08-10", "2020-08-11",
    "2020-08-12", "2020-08-13", "2020-08-14", "2020-08-15",
    "2020-08-16", "2020-08-17", "2020-08-18", "2020-08-19",
    "2020-08-20", "2020-08-21", "2020-08-22", "2020-08-23",
    "2020-08-24", "2020-08-25", "2020-08-26", "2020-08-27",
    "2020-08-28", "2020-08-29", "2020-08-30", "2020-08-31",
    "2020-09-01", "2020-09-02", "2020-09-03", "2020-09-04",
    "2020-09-05", "2020-09-06", "2020-09-07", "2020-09-08",
    "2020-09-09", "2020-09-10", "2020-09-11", "2020-09-12",
    "2020-09-13", "2020-09-14", "2020-09-15", "2020-09-16",
    "2020-09-17", "2020-09-18", "2020-09-19", "2020-09-20",
    "2020-09-21", "2020-09-22", "2020-09-23", "2020-09-24",
    "2020-09-25", "2020-09-26", "2020-09-27", "2020-09-28",
    "2020-09-29", "2020-09-30"
]

epi_dates_large = [
    "2020-06-01", "2020-06-02", "2020-06-03", "2020-06-04", 
    "2020-06-05", "2020-06-06", "2020-06-07", "2020-06-08",
    "2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
    "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
    "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
    "2020-06-21", "2020-06-22", "2020-06-23", "2020-06-24",
    "2020-06-25", "2020-06-26", "2020-06-27", "2020-06-28",
    "2020-06-29", "2020-06-30", "2020-07-01", "2020-07-02",
    "2020-07-03", "2020-07-04", "2020-07-05", "2020-07-06",
    "2020-07-07", "2020-07-08", "2020-07-09", "2020-07-10",
    "2020-07-11", "2020-07-12", "2020-07-13", "2020-07-14",
    "2020-07-15", "2020-07-16", "2020-07-17", "2020-07-18",
    "2020-07-19", "2020-07-20", "2020-07-21", "2020-07-22",
    "2020-07-23", "2020-07-24", "2020-07-25", "2020-07-26",
    "2020-07-27", "2020-07-28", "2020-07-29", "2020-07-30",
    "2020-07-31", "2020-08-01", "2020-08-02", "2020-08-03",
    "2020-08-04", "2020-08-05", "2020-08-06", "2020-08-07",
    "2020-08-08", "2020-08-09", "2020-08-10", "2020-08-11",
    "2020-08-12", "2020-08-13", "2020-08-14", "2020-08-15",
    "2020-08-16", "2020-08-17", "2020-08-18", "2020-08-19",
    "2020-08-20", "2020-08-21", "2020-08-22", "2020-08-23",
    "2020-08-24", "2020-08-25", "2020-08-26", "2020-08-27",
    "2020-08-28", "2020-08-29", "2020-08-30", "2020-08-31",
    "2020-09-01", "2020-09-02", "2020-09-03", "2020-09-04",
    "2020-09-05", "2020-09-06", "2020-09-07", "2020-09-08",
    "2020-09-09", "2020-09-10", "2020-09-11", "2020-09-12",
    "2020-09-13", "2020-09-14", "2020-09-15", "2020-09-16",
    "2020-09-17", "2020-09-18", "2020-09-19", "2020-09-20",
    "2020-09-21", "2020-09-22", "2020-09-23", "2020-09-24",
    "2020-09-25", "2020-09-26", "2020-09-27", "2020-09-28",
    "2020-09-29", "2020-09-30", "2020-10-01", "2020-10-02",
    "2020-10-03", "2020-10-04", "2020-10-05", "2020-10-06",
    "2020-10-07", "2020-10-08", "2020-10-09", "2020-10-10",
    "2020-10-11", "2020-10-12", "2020-10-13", "2020-10-14",
    "2020-10-15", "2020-10-16", "2020-10-17", "2020-10-18",
    "2020-10-19", "2020-10-20", "2020-10-21", "2020-10-22",
    "2020-10-23", "2020-10-24", "2020-10-25", "2020-10-26",
    "2020-10-27", "2020-10-28", "2020-10-29", "2020-10-30",
    "2020-10-31", "2020-11-01", "2020-11-02", "2020-11-03",
    "2020-11-04", "2020-11-05", "2020-11-06", "2020-11-07",
    "2020-11-08", "2020-11-09", "2020-11-10", "2020-11-11",
    "2020-11-12", "2020-11-13", "2020-11-14", "2020-11-15",
    "2020-11-16", "2020-11-17", "2020-11-18", "2020-11-19",
    "2020-11-20", "2020-11-21", "2020-11-22", "2020-11-23",
    "2020-11-24", "2020-11-25", "2020-11-26", "2020-11-27",
    "2020-11-28", "2020-11-29", "2020-11-30", "2020-12-01",
    "2020-12-02", "2020-12-03", "2020-12-04", "2020-12-05",
    "2020-12-06", "2020-12-07", "2020-12-08", "2020-12-09",
    "2020-12-10", "2020-12-11", "2020-12-12", "2020-12-13",
    "2020-12-14", "2020-12-15", "2020-12-16", "2020-12-17",
    "2020-12-18", "2020-12-19", "2020-12-20", "2020-12-21",
    "2020-12-22", "2020-12-23", "2020-12-24", "2020-12-25",
    "2020-12-26", "2020-12-27", "2020-12-28", "2020-12-29",
    "2020-12-30", "2020-12-31"
]

epi_dates_smaller = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
             "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
             ]

epi_dates_one_sample = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15"] # for testing
epi_dates_pred_one_sample = ["2020-06-16"] # for testing

In [ ]:
# THIS IS WITH THE NEW PREPROCESSOR FILE (PREPROCESSOR_FINAL_DATA.PY)

print("Preprocessing data...")

# Preprocess the data
input_hor = 7   # test now with smaller input horizon
pred_hor = 1    # test now with smaller prediction horizon
train_perc, test_perc = 0.8, 0.2

# Preprocess the data
print(os.getcwd())
locations_data = "../final_data/locations_data_unique.npy"
epi_dataset = "../final_data/X_normalized.npy"
preproc = preprocessor_final_data.Preprocessor(flow_dataset, epi_dataset, locations_data, plottable=True)

graph_kronecker_whole_df = preproc.combined_manual_kronecker() # makes the pandas df from the kronecker data

# check for duplicate values in graph_kronecker_whole_df
print("Number of duplicate values in graph_kronecker_whole_df: ", graph_kronecker_whole_df.duplicated().sum())

# print first dataframe in graph_kronecker_whole_df, check if the data is getting pulled correctly
print("Check: First dataframe in graph_kronecker_whole_df: ", graph_kronecker_whole_df.iloc[0])



In [ ]:
# NEWER VERSION

print("Getting the training graph signal...")
# do it first only with cumulative_confirmed
tr_epi = preproc.set_timestep_offset_epi_dataset(from_timestep=0).get_epi_dataset() # will get the epi info for the entire dataset, the entire kronecker, then later we index per training example and pred
print(len(tr_epi)) # working, should be 30
print(tr_epi[0]) # working, should be the geoids of the first day

train_graph_sig = {}
for day in tr_epi:
    entry_count = 0
    for geoid in day.geoid_o:
        if geoid not in train_graph_sig: # which should be for every day
            train_graph_sig[geoid] = {
                'weather1': [],
                'weather2': [],
                'cumulative_confirmed': [],
                'cumulative_deceased': [],
                'new_deceased': [],
                'cumulative_persons_fully_vaccinated': [],
                'new_persons_fully_vaccinated': []
            }

        train_graph_sig[geoid]['weather1'].append(day.weather1[entry_count])
        train_graph_sig[geoid]['weather2'].append(day.weather2[entry_count])
        train_graph_sig[geoid]['cumulative_confirmed'].append(day.cumulative_confirmed[entry_count])
        train_graph_sig[geoid]['cumulative_deceased'].append(day.cumulative_deceased[entry_count])
        train_graph_sig[geoid]['new_deceased'].append(day.new_deceased[entry_count])
        train_graph_sig[geoid]['cumulative_persons_fully_vaccinated'].append(day.cumulative_persons_fully_vaccinated[entry_count])
        train_graph_sig[geoid]['new_persons_fully_vaccinated'].append(day.new_persons_fully_vaccinated[entry_count])
        entry_count += 1
print("Check: Should be number of nodes per day * number of days", len(train_graph_sig))  # working

# sort train_graph_sig by geoid_o
train_graph_sig = dict(sorted(train_graph_sig.items(), key=lambda item: item[0]))

# print the first element of train_graph_sig, check if the data is getting pulled correctly
print("Check: First element in train_graph_sig: ", {k: train_graph_sig[k] for k in list(train_graph_sig)[:1]})




In [ ]:
# NEW CODE 

# Combine all values for each feature across all geo_ids
weather1_values = []
weather2_values = []
cumulative_confirmed_values = []
cumulative_deceased_values = []
new_deceased_values = []
cumulative_persons_fully_vaccinated_values = []
new_persons_fully_vaccinated_values = []

for geoid in train_graph_sig:
    weather1_values.extend(train_graph_sig[geoid]['weather1'])
    weather2_values.extend(train_graph_sig[geoid]['weather2'])
    cumulative_confirmed_values.extend(train_graph_sig[geoid]['cumulative_confirmed'])
    cumulative_deceased_values.extend(train_graph_sig[geoid]['cumulative_deceased'])
    new_deceased_values.extend(train_graph_sig[geoid]['new_deceased'])
    cumulative_persons_fully_vaccinated_values.extend(train_graph_sig[geoid]['cumulative_persons_fully_vaccinated'])
    new_persons_fully_vaccinated_values.extend(train_graph_sig[geoid]['new_persons_fully_vaccinated'])

# Calculate mean and std for each feature across all geo_ids
mean_weather1 = np.mean(weather1_values)
std_weather1 = np.std(weather1_values)

mean_weather2 = np.mean(weather2_values)
std_weather2 = np.std(weather2_values)

mean_cumulative_confirmed = np.mean(cumulative_confirmed_values)
std_cumulative_confirmed = np.std(cumulative_confirmed_values)

mean_cumulative_deceased = np.mean(cumulative_deceased_values)
std_cumulative_deceased = np.std(cumulative_deceased_values)

mean_new_deceased = np.mean(new_deceased_values)
std_new_deceased = np.std(new_deceased_values)

mean_cumulative_persons_fully_vaccinated = np.mean(cumulative_persons_fully_vaccinated_values)
std_cumulative_persons_fully_vaccinated = np.std(cumulative_persons_fully_vaccinated_values)

mean_new_persons_fully_vaccinated = np.mean(new_persons_fully_vaccinated_values)
std_new_persons_fully_vaccinated = np.std(new_persons_fully_vaccinated_values)

# Normalize each feature list in train_graph_sig across all geo_ids
for geoid in train_graph_sig:
    train_graph_sig[geoid]['weather1'] = (train_graph_sig[geoid]['weather1'] - mean_weather1) / std_weather1 if std_weather1 != 0 else 0
    train_graph_sig[geoid]['weather2'] = (train_graph_sig[geoid]['weather2'] - mean_weather2) / std_weather2 if std_weather2 != 0 else 0
    train_graph_sig[geoid]['cumulative_confirmed'] = (train_graph_sig[geoid]['cumulative_confirmed'] - mean_cumulative_confirmed) / std_cumulative_confirmed if std_cumulative_confirmed != 0 else 0
    train_graph_sig[geoid]['cumulative_deceased'] = (train_graph_sig[geoid]['cumulative_deceased'] - mean_cumulative_deceased) / std_cumulative_deceased if std_cumulative_deceased != 0 else 0
    train_graph_sig[geoid]['new_deceased'] = (train_graph_sig[geoid]['new_deceased'] - mean_new_deceased) / std_new_deceased if std_new_deceased != 0 else 0
    train_graph_sig[geoid]['cumulative_persons_fully_vaccinated'] = (train_graph_sig[geoid]['cumulative_persons_fully_vaccinated'] - mean_cumulative_persons_fully_vaccinated) / std_cumulative_persons_fully_vaccinated if std_cumulative_persons_fully_vaccinated != 0 else 0
    train_graph_sig[geoid]['new_persons_fully_vaccinated'] = (train_graph_sig[geoid]['new_persons_fully_vaccinated'] - mean_new_persons_fully_vaccinated) / std_new_persons_fully_vaccinated if std_new_persons_fully_vaccinated != 0 else 0

# print the first element of train_graph_sig, check if the data is getting pulled correctly
print("Check: First element in train_graph_sig: ", {k: train_graph_sig[k] for k in list(train_graph_sig)[:1]})

In [ ]:
all_training_examples = []

adj_kronecker_whole = get_adj_from_plot(graph_kronecker_whole_df)
list_of_geoids = preproc.flow.iloc[:]['geoid_o'].unique()
num_nodes_per_day = len(list_of_geoids)

for example_num in range(0, len(tr_epi) - (input_hor)): # this is how many times you can "shift and have valid data to pull from"
    # now draw out the adjacency matrix per example
    width_of_adj_per_example = num_nodes_per_day*input_hor
    shift = num_nodes_per_day # number of nodes per day

    # this should still work in the new approach
    adj_per_example = adj_kronecker_whole[ (example_num*shift) : (example_num*shift) + width_of_adj_per_example,
                                           (example_num*shift) : (example_num*shift) + width_of_adj_per_example]

    # now drawing out the train_graph_signal per example

    # get the graph signal corresponding to the example nodes
    offset = (example_num * shift)
    train_graph_sig_per_example = {k: train_graph_sig[k] for k in \
                                   list(train_graph_sig)[offset : offset + width_of_adj_per_example]}

    # get the graph signal corresponding to the [input_hor : input_hor+pred_hor] set of nodes -- NEEDS CHECKING
    
    train_graph_sig_per_example_pred = {k: train_graph_sig[k] for k in \
                                        list(train_graph_sig)[offset + (num_nodes_per_day * input_hor) : offset + num_nodes_per_day*(input_hor + pred_hor)]}

    
    training_example = [adj_per_example, train_graph_sig_per_example, train_graph_sig_per_example_pred]
    all_training_examples.append(training_example)

# take the last 20% of the the training examples and use them as test examples
all_test_examples = all_training_examples[int(len(all_training_examples)*train_perc):]
# remove those examples from all_training_examples
all_training_examples = all_training_examples[:int(len(all_training_examples)*train_perc)]


In [ ]:
# store the training_data and testing_data in a pickle file
import pickle
with open("training_data.pkl", "wb") as f:
   pickle.dump(all_training_examples, f)

with open("testing_data.pkl", "wb") as f:
   pickle.dump(all_test_examples, f)

In [ ]:
# load the pickle files
import pickle
with open("training_data.pkl", "rb") as f:
   training_data = pickle.load(f)

with open("testing_data.pkl", "rb") as f:
    testing_data = pickle.load(f)


In [ ]:
# GRAPH CONVOLUTIONAL NETWORK BASED ON KRONECKER GRAPH -- TOO MANY PARAMS NOW, TALK TO PEOPLE ABOUT THIS & HOW TO REDUCE

# Define the model parameters

num_features = len(list(train_graph_sig.values())[0])
input_dim = num_features  # Number of features per node
output_dim = num_features  # Number of output features per node
hidden_dim_low = 70
hidden_dim_high = 150
num_nodes_kron = training_data[0][0].shape[0]
print("Number of nodes in the kronecker graph: ", num_nodes_kron)
num_nodes_pred = len(training_data[0][2])
print(num_nodes_kron)
print(num_nodes_pred)

desired_num_params = 20000

In [ ]:
import Model3
reload(Model3)
from Model3 import GraphConvolutionalNetwork

#  Instantiate the model
input_horizon, prediction_horizon = 0, 0 # TODO amke use of these
model = GraphConvolutionalNetwork(desired_num_params, input_horizon, prediction_horizon, num_features, input_dim, output_dim, num_nodes_kron, num_nodes_pred)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters:", num_params)

# Define the loss function and optimizer
num_epochs = 100
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# shuffle the training data randomly
print( training_data[0][0].shape)
random.shuffle(training_data)
random.shuffle(testing_data)

In [ ]:
# TRAINING WITH NEW PREPROCESSOR
train_losses = []
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    for data in training_data:
        output = model(data) # needs to have shape [num_nodes_pred, num_features]
        # output is of shape [3070] turn into shape [3070, 1] 
        # output = output.view(-1, 1) # I dont think i need because now it is a multidimensional output
        
        loss = model.getLoss(output)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.8f}')



In [ ]:

# Test the model
model.eval()
test_loss = 0.0
predictions = []
targets = []
with torch.no_grad():
    for data in testing_data:
        output = model(data) 

        loss = model.getLoss(output)
        test_loss += loss.item()
    test_loss /= len(testing_data)

# Compute metrics
print(f'Test Loss: {test_loss:.8f}')
print(model.target_graph_signal_matrix[:, 2])


In [ ]:
print(output) # last test output
#print(target_graph_signal) # last test example

# plot the united states and have these values as the nodes

# get a list of the key values in train_graph_sig
geoid_list_tr = list(train_graph_sig.keys())
geoid_list_tar = list(target_graph_signal.keys())

# make a dictionary where key is geoid_list entry and value is out entry
out_dict = {geoid_list_tr[i]: output[i, 2].item() for i in range(len(output))}

target_dict = {i: target_graph_signal[i]['cumulative_confirmed'] for i in geoid_list_tar}

# need to get the geographical information on where to put these nodes
# get the geographical information from the preprocessor, 'geoid_o', 'lat_o', 'lng_o' for all the geoids in out_dict
#print(graph_kronecker_whole_df)

# for each geoid_o which is a key in out_dict, get the lat_o and lng_o from the graph_kronecker_whole_df
# then plot the united states and put the values of out_dict as the node values (new_confirmed)
# then compare with the target_dict values

import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add nodes with attributes
geoid_o = list(out_dict.keys())
lat_o = [graph_kronecker_whole_df[graph_kronecker_whole_df['geoid_o'] == key]['lat_o'].values[0] for key in out_dict.keys()]
lng_o = [graph_kronecker_whole_df[graph_kronecker_whole_df['geoid_o'] == key]['lng_o'].values[0] for key in out_dict.keys()]


# cumulative_confirmed = [out_dict[key] for key in out_dict.keys()] # out_dict[key] will return a dictionary not a value

cumulative_confirmed = []
# pull column 3 from graph_signal_matrix as cumulative_confirmed
for i in range(model.graph_signal_matrix.shape[0]):
    cumulative_confirmed.append(model.graph_signal_matrix[i, 2])


for i in range(len(geoid_o)):
    G.add_node(geoid_o[i], cumulative_confirmed=cumulative_confirmed[i], lat=lat_o[i], lng=lng_o[i])

# Get node positions
pos = {node: (G.nodes[node]['lng'], G.nodes[node]['lat']) for node in G.nodes}

# Get node values
node_values = [G.nodes[node]['cumulative_confirmed'] for node in G.nodes]

# Plot the graph
plt.figure(figsize=(10, 8))
nx.draw_networkx(G, pos, node_color=node_values, cmap='viridis', node_size=node_values, alpha=0.8, with_labels=False)
# plt.colorbar(label='New Confirmed Cases')
plt.title('Graph of COVID-19 Cases: MODEL OUTPUT')
plt.axis('off')
plt.show()



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add nodes with attributes
geoid_o = list(out_dict.keys())
lat_o = [graph_kronecker_whole_df[graph_kronecker_whole_df['geoid_o'] == key]['lat_o'].values[0] for key in out_dict.keys()]
lng_o = [graph_kronecker_whole_df[graph_kronecker_whole_df['geoid_o'] == key]['lng_o'].values[0] for key in out_dict.keys()]


# cumulative_confirmed = [out_dict[key] for key in out_dict.keys()] # out_dict[key] will return a dictionary not a value

cumulative_confirmed = []
# pull column 3 from graph_signal_matrix as cumulative_confirmed
for i in range(model.target_graph_signal_matrix.shape[0]):
    cumulative_confirmed.append(model.target_graph_signal_matrix[i, 2])

for i in range(len(geoid_o)):
    G.add_node(geoid_o[i], cumulative_confirmed=cumulative_confirmed[i], lat=lat_o[i], lng=lng_o[i])

# Get node positions
pos = {node: (G.nodes[node]['lng'], G.nodes[node]['lat']) for node in G.nodes}

# Get node values
node_values = [G.nodes[node]['cumulative_confirmed'] for node in G.nodes]

# Plot the graph
plt.figure(figsize=(10, 8))
nx.draw_networkx(G, pos, node_color=node_values, cmap='viridis', node_size=node_values, alpha=0.8, with_labels=False)
# plt.colorbar(label='New Confirmed Cases')
plt.title('Graph of COVID-19 Cases: Ground Truth')
plt.axis('off')
plt.show()